## Day 7: No Space Left On Device

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

```
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
```

The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

- cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
    - cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
    - cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
    - cd / switches the current directory to the outermost directory, /.
- ls means list. It prints out all of the files and directories immediately contained by the current directory:
    - 123 abc means that the current directory contains a file named abc with size 123.
    - dir xyz means that the current directory contains a directory named xyz.

Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

```
- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
```

Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

- The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
- The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
- Directory d has total size 24933642.
- As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [1]:
#@title Define process_input_file function to read the file and create a file listing
CD_PREFIX = '$ cd '
LS = '$ ls'
DIR = 'dir '

def go_to_parent(path : str) -> str:
    temp_path  = path[:-1]
    temp_path = temp_path[:temp_path.rindex('/')]
    if temp_path != '/':
        temp_path = temp_path + '/'
    return temp_path

def process_input_file(filename: str):
    current_working_directory = ''
    file_listing = []

    with open(filename) as f:
        for line in f:
            trimmed_line = line.rstrip('\n')
            if trimmed_line.startswith(CD_PREFIX):
                if '..' in trimmed_line:
                    current_working_directory = go_to_parent(current_working_directory)
                else: 
                    folder = trimmed_line.replace(CD_PREFIX, '')
                    if folder != '/':
                        folder = folder + '/'
                    current_working_directory = current_working_directory + folder
            elif trimmed_line.startswith(DIR) or trimmed_line == LS: 
                continue
            else:
                file = trimmed_line.split(' ')
                file_name = file[1]
                file_size = file[0]
                new_file_entry = [f'{current_working_directory}{file_name}', int(file_size)]
                file_listing.append(new_file_entry)

    return file_listing

In [2]:
#@title Test process_input_file functions
import unittest

class TestProcessInput(unittest.TestCase):

    def test_process_input_file(self):
        expected = [
            ['/a/e/i', 584],
            ['/a/f', 29116],
            ['/a/g', 2557],
            ['/a/h.lst', 62596],
            ['/b.txt',  14848514],
            ['/c.dat',  8504156],
            ['/d/d.ext', 5626152],
            ['/d/d.log', 8033020],
            ['/d/j', 4060174],
            ['/d/k', 7214296],
        ]

        actual = sorted(process_input_file('test.txt'))
        
        self.assertEqual(actual, expected)
                         
unittest.main(argv=[''], verbosity=2, exit=False)

test_process_input_file (__main__.TestProcessInput.test_process_input_file) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [3]:
#@title Create a function distinct_directories to list all the distinct directories

def distinct_directories(file_listing):
    directories = ['/']
    for file in file_listing:
        temp = ''
        for file_component in file[0].split('/')[1:-1]:
            temp =  temp + '/' + file_component
            directories.append(temp)
        
    return set(directories)

def sum_for_directory(file_listing, dir):
    if dir != '/':
        dir = dir + '/' #force it to be a directory
    return sum([file[1] for file in file_listing if file[0].startswith(dir)])

def directories_under_limit(file_listing, limit):
    total_sum = 0
    for d in distinct_directories(file_listing): 
        sum_dir = sum_for_directory(file_listing, d) 
        if sum_dir <= limit:
            print(f'A {d} {sum_dir}')
            total_sum = total_sum + sum_dir
    
    return total_sum

In [4]:
#@title Test directory functions
import unittest

test_file_listing = [
    ['/a/e/i', 584],
    ['/a/f', 29116],
    ['/a/g', 2557],
    ['/a/h.lst', 62596],
    ['/b.txt',  14848514],
    ['/c.dat',  8504156],
    ['/d/d.ext', 5626152],
    ['/d/d.log', 8033020],
    ['/d/j', 4060174],
    ['/d/k', 7214296],
]

class TestDirectories(unittest.TestCase):
    
    def test_distinct_directories(self):

        expected = set([
            '/',
            '/a',
            '/a/e',
            '/d',
        ])
        
        actual = distinct_directories(test_file_listing)
        
        self.assertEqual(sorted(actual), sorted(expected))

    def test_distict_directories_for_bug(self):
        input = [
            ['/jssbn/pjzpjjq/qlrn/wbgvqpc/qfhvjtv/mhbbpdpj/ghrbbh/tlhttrgs', 24686]
        ]
        expected = set([
            '/',
            '/jssbn',
            '/jssbn/pjzpjjq',
            '/jssbn/pjzpjjq/qlrn',
            '/jssbn/pjzpjjq/qlrn/wbgvqpc',
            '/jssbn/pjzpjjq/qlrn/wbgvqpc/qfhvjtv',
            '/jssbn/pjzpjjq/qlrn/wbgvqpc/qfhvjtv/mhbbpdpj',
            '/jssbn/pjzpjjq/qlrn/wbgvqpc/qfhvjtv/mhbbpdpj/ghrbbh',
        ])
        
        actual = distinct_directories(input)
        
        self.assertEqual(sorted(actual), sorted(expected))

    def test_sum_for_directory(self):
        self.assertEqual(sum_for_directory(test_file_listing, '/'), 48381165)
        self.assertEqual(sum_for_directory(test_file_listing, '/d'), 24933642)
        self.assertEqual(sum_for_directory(test_file_listing, '/a'), 94853)
        self.assertEqual(sum_for_directory(test_file_listing, '/a/e'), 584)        

    def test_sum_for_directory_bug(self):
        my_listing = [
           ['/lfrctthp/zzvnq/lznpqbrd/dvlb.qvr', 175412],
           ['/lfrctthp/zzvnq/lznpqbrd/dvlb/rwr.nbz', 41223],
        ]
        self.assertEqual(sum_for_directory(my_listing, '/lfrctthp/zzvnq/lznpqbrd/dvlb'), 41223)        

    
        
    def test_sum_directories_under_limit(self):
        self.assertEqual(directories_under_limit(test_file_listing, 100000), 95437)
    
unittest.main(argv=[''], verbosity=2, exit=False)

test_distict_directories_for_bug (__main__.TestDirectories.test_distict_directories_for_bug) ... ok
test_distinct_directories (__main__.TestDirectories.test_distinct_directories) ... ok
test_sum_directories_under_limit (__main__.TestDirectories.test_sum_directories_under_limit) ... ok
test_sum_for_directory (__main__.TestDirectories.test_sum_for_directory) ... ok
test_sum_for_directory_bug (__main__.TestDirectories.test_sum_for_directory_bug) ... ok
test_process_input_file (__main__.TestProcessInput.test_process_input_file) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.011s

OK


A /a/e 584
A /a 94853


In [5]:
file_listing = process_input_file('input.txt')

In [6]:
limit = 100000
total_sum = directories_under_limit(file_listing, limit)
print(f'Total sum = {total_sum}')

A /nmzntmcf/scwnmrds/sppmmj/sppmmj 73173
A /jssbn/tlhttrgs/zjghthcb/ghzsqrn/sfhqmzhd 58893
A /lfrctthp/cwjbzd/dfqpqqw 93069
A /jssbn/tlhttrgs/zjghthcb/hvwjc/nchvqtdq 49931
A /jssbn/rbtbtt/wqmvzg/lzbj 5150
A /jssbn/pjzpjjq/dvlb/fnwsmj/lfrctthp/sgddcfdn/lnjln 28970
A /lfrctthp/zzvnq/vnmhs 16118
A /jssbn/rbtbtt/wqmvzg 5150
A /nmzntmcf/scwnmrds 73173
A /lfrctthp/zzvnq/lznpqbrd/dvlb 41223
A /lfrctthp/sqhvvsb/ldmwm/cwjbzd/tlhttrgs/cgf 77864
A /lfrctthp/dvlb/dnqvhtr/jzm/smtpdcpf 95947
A /jssbn/tlhttrgs/zjghthcb/ghzsqrn/mdqbvhrr 32598
A /jssbn/pjzpjjq/qlrn/wbgvqpc/qfhvjtv/mhbbpdpj 24686
A /nmzntmcf/scwnmrds/sppmmj 73173
A /lfrctthp/dvlb/jqpzj 7675
A /nmzntmcf/lvchpdf/tlhttrgs/tlhttrgs/sbhswjld/sppmmj 88100
A /jssbn/pjzpjjq/qlrn/wbgvqpc/qfhvjtv/mhbbpdpj/ghrbbh 24686
A /nmzntmcf/lvchpdf/sppmmj/rdd 20045
A /jssbn/pjzpjjq/dvlb/zhj/nqv/tlhttrgs/cqtnvzn 49609
A /nmzntmcf/lrtsts/drtrdnsg/sppmmj/wwhbgg 51752
A /lfrctthp/cwjbzd/dfqpqqw/rjm 25830
A /jssbn/lfrctthp/lfrctthp 25083
A /jssbn/pjzpjjq/qlrn/wb

## Part Two

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

In the example above, the total size of the outermost directory (and thus the total amount of used space) is 48381165; this means that the size of the unused space must currently be 21618835, which isn't quite the 30000000 required by the update. Therefore, the update still requires a directory with total size of at least 8381165 to be deleted before it can run.

To achieve this, you have the following options:

Delete directory e, which would increase unused space by 584.
Delete directory a, which would increase unused space by 94853.
Delete directory d, which would increase unused space by 24933642.
Delete directory /, which would increase unused space by 48381165.
Directories e and a are both too small; deleting them would not free up enough space. However, directories d and / are both big enough! Between these, choose the smallest: d, increasing unused space by 24933642.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

In [11]:
space_available = 70000000 - sum_for_directory(file_listing, '/')
space_available

29086555

In [12]:
additional_required = 30000000 - space_available
additional_required

913445

In [14]:
totals = []
for d in distinct_directories(file_listing):
    s = sum_for_directory(file_listing, d)
    if s >= additional_required:
        totals.append(s)

print(sorted(totals)[0])

942298
